# San Francisco Crime Classification

From 2010 - 2015 the San Francisco police department made records crime reports from across all of San Francisco's neighborhoods. 

Your task is **to predict if the crime was theft or not!**

![](https://digital.ihg.com/is/image/ihg/holiday-inn-san-francisco-6019097353-2x1)

The following information has been recorded in our data: 
* `Dates` - timestamp of the crime incident
* `Category` - category of the crime incident. 
* `Descript` - detailed description of the crime incident
* `PdDistrict` - name of the Police Department District
* `Resolution` - how the crime incident was resolved
* `Address` - the approximate street address of the crime incident
* `X` - Longitude
* `Y` - Latitude
* `Theft` - A flag that states if the crime was a theft or larcency. **This is the target variable you are going to predict.**

Our target is **Theft**. This column was created from the `Category`column which contains more detailed categories of crime.

This dataset and the corresponding challenge was part of a competition hosted by Kaggle and many people have attempted to solve this problem before you. Take a look at what people have tried at the [Kaggle site](https://www.kaggle.com/c/sf-crime/notebooks).

## Import the libraries

Use this next cell to import the libraries you need, pandas is already done for you. If you need more the further down the notebook you get come back and add them to this cell.

In [ ]:
import pandas as pd

## Steps to follow

We are going to follow the following project steps:

- Import the data
- Exploratory Data Analysis
- In-depth Data Analysis
- Feature Engineering
- Preparing the Data for Sci-kit Learn
- Use a transformer on categorical features
- Build a pipeline
- Using grid search on model parameters
- Analysing model predictive power
- Further improvements
- Conclusion/suggestions for future

## Import the data

First import the data from the `sf_crime_hackathon.csv` in the `/data` folder.

In [ ]:
crime = pd.read_csv('data/sf_crime_hackathon.csv')

## Data exploration

### Part 1: Exploratory Analysis
Have a quick look at the data using these methods:

- `df.head()`, `df.tail()`, `df.sample(5)`
- `df.shape`
- `df.describe()`
- `df.info()`
- `df.dtypes`

You may want to clean the data a bit before moving on.

Re-read the data in and add parameters to `.read_csv()` and chain methods to:
- parse the type of the `dates` column: make sure they are datetimes instead of strings;
- rename the colums to be all lower case (or upper if you prefer);
- rename the column `dates` to be `date`.

### Part 2: In-depth Analysis (with optional visualisations)

Answer the following questions. You can also visualise your results using `.plot()` if you want. 

1. Have a look at the different columns in the data. Which ones do you have access to now, but not once the model is actually being used? What should you do when training the model?

2. How many missing values are there in the data?

3. How many unique values for address are there?

4. What are the top 10 largest categories?

5. Which month has the most amount of data?

6. Which district has the most amount of crime? Which has the least?

7. What are the 10 most/least occuring crimes?

# Feature Engineering

Now you're going to create some new features for our model. Let's start by creating new features from our `date` feature. If you converted this column to a Timestamp, you can easily extract several [date properties](https://pandas.pydata.org/docs/reference/arrays.html#properties)!


Create the following columns and drop the original `date` column:
- `n_days`: Number of days since the first date in the dataset
- `day`: The day of the year
- `weekday`: The day of the week
- `month`: The month of the year
- `year`: The year
- `hour`: The hour of the day
- `minute`: The minute of the hour


Make sure the features have been created correctly. What is the max and min of the new features? Is that what you expect?

Now let's make some extra features. The address column will be an issue since there are over 22,000 unique values. You also probably want to drop such sensitive data. Instead let's create:
- `is_block`: Take the value of True if the `address` feature has the word `Block` in
- `x_minus_y`: The difference between `x` and `y`
- `x_plus_y`: The sum of `x` and `y`

## Bonus

Add at least 2 extra features.

Potential features:

- Any special date fields
- Features based on past observations (e.g. crime in the same area during the past year)
- Converting categorical features (`pddistrict`) into numeric using target encoding
- Any other combination of features
- External features (from other datasets)

# Prepare the data for sci-kit learn

First you will need to separate the features and target.

Note that two features will need to be dropped:
- `descript` - detailed description of the crime incident, therefore a more detailed version of the **target feature**
- `resolution` - how the crime incident was resolved. This is created after the target was defined, and cannot be used to predict a new crime.

Create X and y. 

- X should have the following features (plus whatever bespoke ones you've created): 
```python
'pddistrict', 'x', 'y','n_days', 'day', 'dayofweek', 'month', 'year', 'hour', 'minute', 'x_minus_y', 'x_plus_y', 'is_block'
```
- y should only have one feature: `theft`

Now let's look at which features are categorical (`object` data types):

Looks like we have some non-numeric data that you will need to change later! (`pddistrict`, `address`)

First let's split into train and test:

# Build a base-line model

Choose a model to build and use the raw data to build a base-line model. Drop all missing rows.

How does your model perform? How do other algorithms perform? Why are some better than others?

# Encoding

You can use one-hot encoding to convert the categorical features, or just drop them for now while building a baseline model.


### Add ColumnTransformer

Let's include `ColumnTransformer()` so that we can apply the onehotencoder to only the categorical columns.

# Create a pipeline

Now that we know we have to transform our data using a sklearn transformer, it would be good to package this up into a pipeline.

Let's build a RandomForest to begin with.

# Gridsearch

Now let's see if you can hypertune these parameters!

# Metrics

So far you've looked at Accuracy Score, F1-Score, Recall and Precision. Can you create an output that reports these metrics?

# Feature Selection

Now that you have your model, can you look at the feature importance to reduce the features in your model and still maintain the score?

To access the feature importance you must:

1. access the model from the pipeline: `pipeline['model']`
2. access the feature importances attribute using `.feature_importances_`

You can then plot this against the feature names (`.get_feature_names_`)

## Rebuild the model using fewer characteristics